In [52]:
import numpy as np
import pandas as pd
from scipy.spatial import cKDTree


In [53]:
pm25_2024 = pd.read_csv("new_data/mappm252024g.csv")
msoa = pd.read_csv("new_data/infuse_msoa_lyr_2011.csv")
pm25_2024.head(10)

/tmp/ipykernel_1948/4006805598.py:1: DtypeWarning: Columns (0,1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  pm25_2024 = pd.read_csv("new_data/mappm252024g.csv")


,pm2.5,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,2024,NaN,NaN,NaN
1,annual mean,NaN,NaN,NaN
2,ug m-3,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,gridcode,x,y,pm252024g
5,55671,460500,1219500,2.995311
6,56360,459500,1218500,2.990555
7,56361,460500,1218500,2.996322
8,56362,461500,1218500,2.997415
9,56363,462500,1218500,2.998089


In [54]:
pm25_2024.drop(labels=[0,1,2,3,4], inplace=True)

pm25_2024.rename(columns={"pm2.5": "ukgridcode", "Unnamed: 1": "x", "Unnamed: 2": "y", "Unnamed: 3": "pm25"}, inplace=True)

pm25_2024.head(10)

,ukgridcode,x,y,pm25
5,55671,460500,1219500,2.995311
6,56360,459500,1218500,2.990555
7,56361,460500,1218500,2.996322
8,56362,461500,1218500,2.997415
9,56363,462500,1218500,2.998089
10,56364,463500,1218500,2.999147
11,57050,459500,1217500,2.99308
12,57051,460500,1217500,2.999035
13,57052,461500,1217500,3.001105
14,57053,462500,1217500,3.00137


In [55]:
pm25_2024["pm25"] = pd.to_numeric(pm25_2024["pm25"], errors="coerce")
pm25_2024 = pm25_2024.astype({"x": "int", "y": "int"})
pm25_2024.head(10)

,ukgridcode,x,y,pm25
5,55671,460500,1219500,2.995311
6,56360,459500,1218500,2.990555
7,56361,460500,1218500,2.996322
8,56362,461500,1218500,2.997415
9,56363,462500,1218500,2.998089
10,56364,463500,1218500,2.999147
11,57050,459500,1217500,2.993080
12,57051,460500,1217500,2.999035
13,57052,461500,1217500,3.001105
14,57053,462500,1217500,3.001370


In [56]:
col3 = pm25_2024.columns[3]
pm25_2024[col3] = pm25_2024[col3].replace(["MISSING", "pm252024g", "NaN"], 0)
pm25_2024.head(10)

,ukgridcode,x,y,pm25
5,55671,460500,1219500,2.995311
6,56360,459500,1218500,2.990555
7,56361,460500,1218500,2.996322
8,56362,461500,1218500,2.997415
9,56363,462500,1218500,2.998089
10,56364,463500,1218500,2.999147
11,57050,459500,1217500,2.993080
12,57051,460500,1217500,2.999035
13,57052,461500,1217500,3.001105
14,57053,462500,1217500,3.001370


In [57]:
pm25_2024.sort_values(by=pm25_2024.columns[3], ascending=False).head(10)

,ukgridcode,x,y,pm25
217004,771963,532500,181500,11.27488
217002,771961,530500,181500,11.10024
216729,771285,544500,182500,11.05037
217003,771962,531500,181500,11.01223
217001,771960,529500,181500,10.97158
217005,771964,533500,181500,10.71323
216717,771273,532500,182500,10.70021
217280,772650,529500,180500,10.56973
216716,771272,531500,182500,10.53258
217281,772651,530500,180500,10.52863


# Break

In [60]:
pollution_coords = np.vstack((pm25_2024["x"], pm25_2024["y"])).T
pollution_tree = cKDTree(pollution_coords)

msoa_coords = np.vstack((msoa["x"], msoa["y"])).T
dist, idx = pollution_tree.query(msoa_coords, k=1)

msoa["pm25_2024"] = pm25_2024.iloc[idx]["pm25"].values

In [61]:
msoa.sort_values(by="pm25_2024", ascending=False).head()

,geo_code,geo_label,geo_labelw,label,name,x,y,pm25_2004,pm25_2024
480,E02000001,City of London 001,NaN,E92000001E41000324E02000001,City of London 001,532464.075463,181219.687532,11.27488,11.27488
5386,E02000193,Camden 028,NaN,E92000001E09000007E02000193,Camden 028,530366.704884,181622.898132,11.10024,11.10024
2234,E02000192,Camden 027,NaN,E92000001E09000007E02000192,Camden 027,531013.533652,181972.825529,11.01223,11.01223
413,E02000972,Westminster 013,NaN,E92000001E41000324E02000972,Westminster 013,529177.048674,181229.161934,10.97158,10.97158
6051,E02000878,Tower Hamlets 015,NaN,E92000001E09000030E02000878,Tower Hamlets 015,533892.136661,181772.807128,10.71323,10.71323


In [62]:
msoa.loc[msoa["geo_code"] == "E02000001"].head()

,geo_code,geo_label,geo_labelw,label,name,x,y,pm25_2004,pm25_2024
480,E02000001,City of London 001,NaN,E92000001E41000324E02000001,City of London 001,532464.075463,181219.687532,11.27488,11.27488
